In [18]:
# Cell 1: Install dependencies & imports
#@title Install & Import
!pip install --quiet pandas numpy scikit-learn

import pandas as pd
import numpy as np
from google.colab import files


# **1: Imports & Argument Parsing**

# **2: Load LOB Events**

In [20]:
# Cell 2: Upload your CSV(s)
#@title Upload LOB Event Data
# Upload `first_25000_rows.csv` (and any other CSVs for cross-asset OFI).
uploaded = files.upload()
print("Uploaded files:", list(uploaded.keys()))


Saving first_25000_rows.csv to first_25000_rows (2).csv
Uploaded files: ['first_25000_rows (2).csv']


In [7]:
# Cell 3: Load the event feed
#@title Define load_lob_events()
def load_lob_events(path: str) -> pd.DataFrame:
    """
    Load a LOB event CSV and index by the event timestamp.
    Expects columns:
      ts_event  (ISO datetime), action {A,C,T}, side {B,A,N},
      depth (0=best,1=2nd,…), price, size.
    """
    df = pd.read_csv(path, parse_dates=['ts_event'])
    df = df.sort_values('ts_event').set_index('ts_event')
    # map depth→level for clarity
    df['level'] = df['depth'] + 1
    return df

# Quick check
lob = load_lob_events('first_25000_rows.csv')
print("Events:", lob.shape)
lob.head()


Events: (5000, 74)


,ts_recv,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,...,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol,level
ts_event,,,,,,,,,,,,,,,,,,,,,
2024-10-21 11:54:29.221064336+00:00,2024-10-21T11:54:29.221230963Z,10,2,38,C,B,1,233.62,2,130,...,1,7,233.25,234.13,55,400,2,1,AAPL,2
2024-10-21 11:54:29.223769812+00:00,2024-10-21T11:54:29.223936626Z,10,2,38,A,B,0,233.67,2,130,...,1,7,233.25,234.13,55,400,2,1,AAPL,1
2024-10-21 11:54:29.225030400+00:00,2024-10-21T11:54:29.225196809Z,10,2,38,A,B,0,233.67,3,130,...,1,7,233.25,234.13,55,400,2,1,AAPL,1
2024-10-21 11:54:29.712434212+00:00,2024-10-21T11:54:29.712600612Z,10,2,38,A,B,2,233.52,200,130,...,1,7,233.25,234.13,55,400,2,1,AAPL,3
2024-10-21 11:54:29.764673165+00:00,2024-10-21T11:54:29.764839221Z,10,2,38,C,B,2,233.52,200,130,...,1,7,233.25,234.13,55,400,2,1,AAPL,3


# **3: Compute Per-Event OFI**

In [21]:
# Cell 3: Set file paths & parameters
#@title Define input file, cross files, interval & levels

# Primary file (exact name from upload)
input_file = 'first_25000_rows.csv'

# Any other uploaded files for cross-asset OFI
cross_files = [f for f in uploaded.keys() if f != input_file]

# Time bucket size (seconds) and number of depth levels
interval = 60
levels   = 5

print("Primary LOB file:", input_file)
print("Cross-asset files:", cross_files)
print("Interval (s):", interval, "Levels:", levels)


Primary LOB file: first_25000_rows.csv
Cross-asset files: ['first_25000_rows (2).csv']
Interval (s): 60 Levels: 5


# **4: Best-Level OFI**

In [22]:
# Cell 4: Load LOB event feed
#@title Define load_lob_events()

def load_lob_events(path: str) -> pd.DataFrame:
    """
    Load a LOB event CSV into a DataFrame.
    Expects columns:
      - ts_event  : ISO datetime
      - action    : 'A', 'C', or 'T'
      - side      : 'B', 'A', or 'N'
      - depth     : 0 = best level, 1 = second level, ...
      - price     : quote price
      - size      : order size
    Returns a DataFrame indexed by ts_event, with extra 'level'=depth+1.
    """
    df = pd.read_csv(path, parse_dates=['ts_event'])
    df = df.sort_values('ts_event').set_index('ts_event')
    df['level'] = df['depth'] + 1
    return df

# Load primary LOB
lob = load_lob_events(input_file)
print("Loaded LOB events:", lob.shape)
lob.head()


Loaded LOB events: (5000, 74)


,ts_recv,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,...,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol,level
ts_event,,,,,,,,,,,,,,,,,,,,,
2024-10-21 11:54:29.221064336+00:00,2024-10-21T11:54:29.221230963Z,10,2,38,C,B,1,233.62,2,130,...,1,7,233.25,234.13,55,400,2,1,AAPL,2
2024-10-21 11:54:29.223769812+00:00,2024-10-21T11:54:29.223936626Z,10,2,38,A,B,0,233.67,2,130,...,1,7,233.25,234.13,55,400,2,1,AAPL,1
2024-10-21 11:54:29.225030400+00:00,2024-10-21T11:54:29.225196809Z,10,2,38,A,B,0,233.67,3,130,...,1,7,233.25,234.13,55,400,2,1,AAPL,1
2024-10-21 11:54:29.712434212+00:00,2024-10-21T11:54:29.712600612Z,10,2,38,A,B,2,233.52,200,130,...,1,7,233.25,234.13,55,400,2,1,AAPL,3
2024-10-21 11:54:29.764673165+00:00,2024-10-21T11:54:29.764839221Z,10,2,38,C,B,2,233.52,200,130,...,1,7,233.25,234.13,55,400,2,1,AAPL,3


# **5: Multi-Level OFI**

In [23]:
# Cell 5: Compute event-level OFI
#@title Define compute_event_ofi() & attach to lob

def compute_event_ofi(row: pd.Series) -> float:
    """
    Compute one event's Order Flow contribution:
      - Add bid  (A,B): +size
      - Cancel bid (C,B): -size
      - Add ask  (A,A): -size
      - Cancel ask (C,A): +size
      - Others (T trades, neutral): 0
    """
    if row.action == 'A' and row.side == 'B':
        return  row.size
    if row.action == 'C' and row.side == 'B':
        return -row.size
    if row.action == 'A' and row.side == 'A':
        return -row.size
    if row.action == 'C' and row.side == 'A':
        return  row.size
    return 0.0

# Apply to all events
lob['event_ofi'] = lob.apply(compute_event_ofi, axis=1)
print("event_ofi stats:")
print(lob['event_ofi'].describe())


event_ofi stats:
count    5000.00000
mean        2.23480
std        71.78672
min       -74.00000
25%       -74.00000
50%         0.00000
75%        74.00000
max        74.00000
Name: event_ofi, dtype: float64


# ***6: Integrated OFI***

In [24]:
# Cell 6: Define OFI feature functions
#@title best_level_ofi(), multi_level_ofi(), integrated_ofi(), cross_asset_ofi()

def best_level_ofi(df: pd.DataFrame, level: int, interval: int) -> pd.Series:
    """
    For depth `level`, sum df['event_ofi'] over non-overlapping
    buckets of `interval` seconds.
    """
    sel = df.loc[df['level']==level, 'event_ofi']
    return sel.resample(f'{interval}S').sum().fillna(0.0)

def multi_level_ofi(df: pd.DataFrame, levels: int, interval: int) -> pd.DataFrame:
    """
    Build a DataFrame of shape (n_buckets, levels)
    with columns ofi_1 ... ofi_levels.
    """
    return pd.DataFrame({
        f'ofi_{lvl}': best_level_ofi(df, lvl, interval)
        for lvl in range(1, levels+1)
    })

def integrated_ofi(ofi_df: pd.DataFrame) -> pd.Series:
    """
    Compute Integrated OFI as a weighted sum:
      weight_k = 1.0 / k
    """
    wtd = ofi_df.copy()
    for col in ofi_df.columns:
        k = int(col.split('_')[1])
        wtd[col] *= (1.0 / k)
    return wtd.sum(axis=1).rename('ofi_integrated')

def cross_asset_ofi(main: pd.Series, others: list) -> pd.Series:
    """
    cross_ofi(t) = sum_j OFI_j(t) - main_ofi(t)
    """
    if not others:
        return pd.Series(name='cross_ofi', dtype=float)
    df = pd.concat([main] + others, axis=1).fillna(0.0)
    return (df.sum(axis=1) - df.iloc[:,0]).rename('cross_ofi')


# **7: Cross-Asset OFI**

In [25]:
# Cell 7: Compute all OFI features
#@title Compute best_ofi, multi-level, integrated, cross-asset

# Best-Level OFI
best1 = best_level_ofi(lob, level=1, interval=interval)
print("best_ofi head:\n", best1.head(), "\n")

# Multi-Level OFI
ml = multi_level_ofi(lob, levels=levels, interval=interval)
print("multi-level OFI head:\n", ml.head(), "\n")

# Integrated OFI
integ = integrated_ofi(ml)
print("integrated OFI head:\n", integ.head(), "\n")

# Cross-Asset OFI (if any)
others = []
for fn in cross_files:
    df_o = load_lob_events(fn)
    df_o['event_ofi'] = df_o.apply(compute_event_ofi, axis=1)
    others.append(best_level_ofi(df_o, level=1, interval=interval))
cross = cross_asset_ofi(best1, others)
if not cross.empty:
    print("cross_ofi head:\n", cross.head(), "\n")


<ipython-input-24-bb532b57d77b>:10: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  return sel.resample(f'{interval}S').sum().fillna(0.0)


best_ofi head:
 ts_event
2024-10-21 11:54:00+00:00     74.0
2024-10-21 11:55:00+00:00   -370.0
2024-10-21 11:56:00+00:00     74.0
2024-10-21 11:57:00+00:00    148.0
2024-10-21 11:58:00+00:00    296.0
Freq: 60s, Name: event_ofi, dtype: float64 

multi-level OFI head:
                            ofi_1  ofi_2  ofi_3  ofi_4  ofi_5
ts_event                                                    
2024-10-21 11:54:00+00:00   74.0    0.0    0.0  -74.0   74.0
2024-10-21 11:55:00+00:00 -370.0   74.0  518.0  -74.0    0.0
2024-10-21 11:56:00+00:00   74.0    0.0    0.0  -74.0    0.0
2024-10-21 11:57:00+00:00  148.0  -74.0  296.0 -222.0 -222.0
2024-10-21 11:58:00+00:00  296.0    0.0  222.0    0.0 -148.0 

integrated OFI head:
 ts_event
2024-10-21 11:54:00+00:00     70.300000
2024-10-21 11:55:00+00:00   -178.833333
2024-10-21 11:56:00+00:00     55.500000
2024-10-21 11:57:00+00:00    109.766667
2024-10-21 11:58:00+00:00    340.400000
Freq: min, Name: ofi_integrated, dtype: float64 

cross_ofi head:
 ts_ev

<ipython-input-24-bb532b57d77b>:10: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  return sel.resample(f'{interval}S').sum().fillna(0.0)


# **8: Assemble & Write Out**

In [27]:
# Cell 8: Assemble & Download Final OFI Output
#@title Assemble & Download OFI Output (with fillna)

# 1. Collect into dict
cols = {
    'best_ofi': best1,
    **{c: ml[c] for c in ml.columns},
    'ofi_integrated': integ
}
if not cross.empty:
    cols['cross_ofi'] = cross

# 2. Build DataFrame and fill any missing values with 0
result = pd.DataFrame(cols).fillna(0.0)

# 3. Sanity check — now there should be no NaNs
assert not result.isna().any().any(), "Still found NaNs!"

# 4. Save & trigger download
result.to_csv('ofi_output.csv')
files.download('ofi_output.csv')

print(" ofi_output.csv is ready (all NaNs filled with 0)")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 ofi_output.csv is ready (all NaNs filled with 0)


# Task 1: Conceptual Questions & Answers

**1. What’s the motivation behind measuring OFI at multiple depth levels of the order book?**  
Measuring beyond the top‐of‐book captures hidden liquidity and replenishment dynamics when best quotes are consumed. Aggregating liquidity changes at deeper levels reveals supply/demand pressure that the best‐level alone misses, improving explanation of short‐term price moves.

**2. Why do the authors use Lasso regression rather than OLS for estimating cross‐impact?**  
Cross‐impact models feature many assets (high‐dimensional predictors) with strong collinearity. Lasso’s ℓ₁ penalty both shrinks coefficients and enforces sparsity, mitigating overfitting and selecting only the most impactful cross‐asset terms.

**3. Why is OFI considered a better predictor of short‐term returns than trade volume?**  
Trade volume aggregates executed trades without distinguishing passive vs. aggressive orders or their sign. OFI directly tracks changes in resting bid/ask sizes—net supply/demand pressure—giving a cleaner, more immediate signal of directional price pressure.


# Task 2: SOR Reading Prep

Please review **Cont & Kukanov (2014) “Optimal Order Placement in Limit Order Markets”** and be ready to discuss:

- **Problem formulation:** objective function (execution cost + penalties) & constraints.  
- **Single‐exchange closed‐form:** how size splits between market vs. limit orders via inverse‐CDF rule.  
- **Multi‐exchange algorithm:** stochastic approximation (Robbins–Monro) with gradient sampling & projection.  
- **Microstructure inputs:** queue sizes \(Q_k\), outflow \(\xi_k\), rebates \(r_k\) in routing decisions.
